# Consommer la GeoAPI pour Pandas/Geopandas

Actuellement, l'Etat met à disposition des API pour obtenir des informations sur les communes, départements et régions de Métropole et des DROM.

Pour les consommer, un certain nombre d'informations est disponible sur https://api.gouv.fr/explorer/geoapi/
Si vous faites, un usage massif, merci de prévenir les gestionnaires comme suggéré dans les informations du lien ci-dessus.

Il existe un package R pour consommer la GeoAPI http://colinfay.me/rgeoapi/ pour information.


In [120]:
# Import pour l'ensemble du notebook Python
try:
    from urllib.parse import urlencode, quote
except ImportError:
    from urllib import urlencode, quote

import json
import unicodedata
import requests
import yaml
from IPython.display import HTML
from pandas import pandas as pd
from geopandas import geopandas as gpd

In [121]:
# Toutes les régions (code + nom) depuis l'API
df = pd.read_json('https://geo.api.gouv.fr/regions')

print(df['code'].dtype, df['nom'].dtype)

HTML(df.to_html())

(dtype('int64'), dtype('O'))


,code,nom
0,1,Guadeloupe
1,2,Martinique
2,3,Guyane
3,4,La Réunion
4,6,Mayotte
5,11,Île-de-France
6,24,Centre-Val de Loire
7,27,Bourgogne-Franche-Comté
8,28,Normandie
9,32,Nord-Pas-de-Calais-Picardie


In [122]:
# Filtrage des régions par nom
params = urlencode({"nom": "loire"})

encoded_url = 'https://geo.api.gouv.fr/regions?%s' % params
df1 = pd.read_json(encoded_url)

HTML(df1.to_html())

,_score,code,nom
0,0.494226,52,Pays de la Loire
1,0.467545,24,Centre-Val de Loire


In [123]:
# Region par code
req = requests.get('http://geo.api.gouv.fr/regions/24')
df2 = pd.DataFrame([req.json()])
HTML(df2.to_html())

,code,nom
0,24,Centre-Val de Loire


In [124]:
# Départements d'une région
df2 = pd.read_json('https://geo.api.gouv.fr/regions/52/departements')
HTML(df2.to_html())

,code,codeRegion,nom
0,44,52,Loire-Atlantique
1,49,52,Maine-et-Loire
2,53,52,Mayenne
3,72,52,Sarthe
4,85,52,Vendée


In [125]:
# Liste des départements
df3 = pd.read_json('https://geo.api.gouv.fr/departements')

HTML(df3.to_html())

,code,codeRegion,nom
0,01,84,Ain
1,02,32,Aisne
2,03,84,Allier
3,04,93,Alpes-de-Haute-Provence
4,05,93,Hautes-Alpes
5,06,93,Alpes-Maritimes
6,07,84,Ardèche
7,08,44,Ardennes
8,09,76,Ariège
9,10,44,Aube


In [126]:
# Informations d'un département
req = requests.get('https://geo.api.gouv.fr/departements/44?fields=nom,code,codeRegion,region')
filtered = req.json()
filtered['nomRegion'] = filtered['region']['nom']
filtered.pop('region', None)

df2 = pd.DataFrame([filtered])
HTML(df2.to_html())

,code,codeRegion,nom,nomRegion
0,44,52,Loire-Atlantique,Pays de la Loire


In [127]:
# Communes appartenant à un département
params = {
    'fields': ','.join(['nom', 'code', 'centre', 'surface', 'codeDepartement', 'codeRegion', 'population']),
    'format': 'json',
    'geometry': 'centre'
}

req = requests.get('https://geo.api.gouv.fr/departements/44/communes?%s' % urlencode(params))
communes = req.json()
for commune in communes:
    commune['x'] = commune['centre']['coordinates'][0]
    commune['y'] = commune['centre']['coordinates'][1]
    commune.pop('centre', None)

df2 = pd.DataFrame(communes)
HTML(df2.to_html())

,code,codeDepartement,codeRegion,nom,population,surface,x,y
0,44001,44,52,Abbaretz,1984,6232,-1.495055,47.552699
1,44002,44,52,Aigrefeuille-sur-Maine,3516,1470,-1.413858,47.074195
2,44003,44,52,Ancenis,7474,2004,-1.178178,47.382618
3,44163,44,52,Vair-sur-Loire,4491,4937,-1.117662,47.408449
4,44005,44,52,Chaumes-en-Retz,6503,7764,-1.919205,47.145218
5,44006,44,52,Assérac,1797,3381,-2.415742,47.432895
6,44007,44,52,Avessac,2542,7640,-1.960712,47.640744
7,44029,44,52,Divatte-sur-Loire,6595,3559,-1.316763,47.285980
8,44009,44,52,Basse-Goulaine,8361,1379,-1.460538,47.207748
9,44010,44,52,Batz-sur-Mer,3008,1043,-2.469416,47.280766


In [128]:
# Communes appartenant à un département avec géométrie point et retour GeoJSON
params = {
    'fields': ','.join(['nom', 'code', 'surface', 'codeDepartement', 'codeRegion', 'population']),
    'format': 'geojson',
    'geometry': 'centre'
}

req = requests.get('https://geo.api.gouv.fr/departements/44/communes?%s' % urlencode(params))
communes = req.json()

gdf1 = gpd.GeoDataFrame.from_features(communes['features'])
HTML(gdf1.to_html())

,code,codeDepartement,codeRegion,geometry,nom,population,surface
0,44001,44,52,POINT (-1.49505512783497 47.55269872437454),Abbaretz,1984,6232
1,44002,44,52,POINT (-1.413857807423431 47.07419473881576),Aigrefeuille-sur-Maine,3516,1470
2,44003,44,52,POINT (-1.178177611876991 47.38261768284986),Ancenis,7474,2004
3,44163,44,52,POINT (-1.117662238939072 47.40844931335979),Vair-sur-Loire,4491,4937
4,44005,44,52,POINT (-1.919205150833175 47.1452183290674),Chaumes-en-Retz,6503,7764
5,44006,44,52,POINT (-2.415742331676649 47.43289470284589),Assérac,1797,3381
6,44007,44,52,POINT (-1.960711919955419 47.64074362173093),Avessac,2542,7640
7,44029,44,52,POINT (-1.316763199180767 47.28598024134344),Divatte-sur-Loire,6595,3559
8,44009,44,52,POINT (-1.460538199526928 47.20774847440808),Basse-Goulaine,8361,1379
9,44010,44,52,POINT (-2.469415946384689 47.28076586999089),Batz-sur-Mer,3008,1043


In [129]:
# Communes appartenant à un département avec géométrie polygone ("contour" dans l'API) et retour GeoJSON
params = {
    'fields': ','.join(['nom', 'code', 'surface', 'codeDepartement', 'codeRegion', 'population']),
    'format': 'geojson',
    'geometry': 'contour'
}

req = requests.get('https://geo.api.gouv.fr/departements/44/communes?%s' % urlencode(params))
communes = req.json()

gdf2 = gpd.GeoDataFrame.from_features(communes['features'])
HTML(gdf2.to_html())

,code,codeDepartement,codeRegion,geometry,nom,population,surface
0,44001,44,52,"POLYGON ((-1.570887710264 47.53602081910284, -...",Abbaretz,1984,6232
1,44002,44,52,POLYGON ((-1.441132984500623 47.09310707072733...,Aigrefeuille-sur-Maine,3516,1470
2,44003,44,52,POLYGON ((-1.231324083709179 47.39239481101578...,Ancenis,7474,2004
3,44163,44,52,"POLYGON ((-1.16384829895241 47.40990438781157,...",Vair-sur-Loire,4491,4937
4,44005,44,52,"POLYGON ((-1.9969846108336 47.19727701242018, ...",Chaumes-en-Retz,6503,7764
5,44006,44,52,POLYGON ((-2.457662697254551 47.44790118052646...,Assérac,1797,3381
6,44007,44,52,POLYGON ((-2.012193986909028 47.66657028631911...,Avessac,2542,7640
7,44029,44,52,POLYGON ((-1.402215900267412 47.28157927133872...,Divatte-sur-Loire,6595,3559
8,44009,44,52,POLYGON ((-1.480987550564341 47.22633571260396...,Basse-Goulaine,8361,1379
9,44010,44,52,(POLYGON ((-2.500648700567182 47.2885347806702...,Batz-sur-Mer,3008,1043


In [130]:
# Alternative en utilisant Swagger
req = requests.get('https://github.com/sgmap/api-communes/raw/master/definition.yml')

content = yaml.load(req.content)

# On enlève les accents dans les titres et les tags car le code generator de swagger
# supprime les lettres avec accents plutôt que les passer en équivalent ascii

# On enlève les accents du titre
content['info']['title'] = unicodedata.normalize('NFD', unicode(content['info']['title'])).encode('ascii', 'ignore')
# On boucle sur les chemins et les méthodes HTTP
for path_key in content['paths'].keys():
    for method in content['paths'][path_key].keys():
        tags = content['paths'][path_key][method]['tags']
        # On ne garde qu'un tag pour ne pas dupliquer dans l'API générée les points d'entrées 
        if len(tags) > 1:
            content['paths'][path_key][method]['tags'] = tags[:1]
        # On enlève les accents
        if len(content['paths'][path_key][method]['tags']) == 1:
            content['paths'][path_key][method]['tags'][0] = unicodedata.normalize('NFD', unicode(content['paths'][path_key][method]['tags'][0])).encode('ascii', 'ignore')

with open('definition.json','wb') as f:
    json.dump(content, f, indent=4)


### Générer un client JavaScript

```
java -jar modules/swagger-codegen-cli/target/swagger-codegen-cli.jar generate -i /home/thomasg/git/geoapi-data-gouv-fr/definition.json -l javascript -o /tmp/geoapi-client/javascript
```

### Générer un client Python
```
java -jar modules/swagger-codegen-cli/target/swagger-codegen-cli.jar generate -i /home/thomasg/git/geoapi-data-gouv-fr/definition.json -l python -o /tmp/geoapi-client/python
```